Load Excel file

In [45]:
import pandas as pd
from pathlib import Path
import os
import unicodedata

In [46]:
BASE_DIR = Path().resolve() 

In [47]:
BASE_DIR

WindowsPath('C:/Users/nicol/OneDrive/Documentos/GitHub/timeTablingFisi/Extraccion')

In [48]:
CURRENT_DIR=(BASE_DIR / ".." / "Data" / "Origen" / "DisponibilidadDocente").resolve()

In [49]:
CURRENT_DIR

WindowsPath('C:/Users/nicol/OneDrive/Documentos/GitHub/timeTablingFisi/Data/Origen/DisponibilidadDocente')

# Buscar todos los archivos .xlsx y .xls

In [50]:
excel_files = list(CURRENT_DIR.glob("*.xlsx")) + list(CURRENT_DIR.glob("*.xls"))

# Leer cada archivo y guardarlo en una lista de DataFrames

In [51]:
nombres_archivos = [file.name for file in excel_files]

In [52]:
nombres_archivos

['Aguilar.xlsx',
 'Armas.xlsx',
 'Bayona.xlsx',
 'Jacinto.xlsx',
 'Lam.xlsx',
 'Moquillaza.xlsx']

In [53]:
## Uso OS
def procesar_archivo(filepath):
    xls = pd.ExcelFile(filepath)
    df = xls.parse(xls.sheet_names[0], header=None)

    # Dataset A (fila 6 → índice 5)
    fila_a = df.iloc[5, [0, 1, 2, 3, 5, 7]]
    dataset_a = pd.DataFrame([fila_a.values], columns=["CodDocente", "ApellidoPaterno", "ApellidoMaterno", "Nombres", "Categoria/Clase", "Hora"])
    cod_docente = fila_a[0]

    # Dataset B (fila 9 a 23 → índice 8:23)
    dataset_b_raw = df.iloc[8:23, 0:8].copy()
    dataset_b_raw.columns = ["Hora", "Lun", "Mar", "Mie", "Jue", "Vie", "Sab", "Dom"]
    dataset_b_raw.reset_index(drop=True, inplace=True)

    # Convertir X en rangos de disponibilidad
    disponibilidad = []
    for dia in ["Lun", "Mar", "Mie", "Jue", "Vie", "Sab", "Dom"]:
        hora_inicio = None
        for i, row in dataset_b_raw.iterrows():
            try:
                hora = int(row["Hora"])
            except:
                continue  # saltar filas como 'HORA' o vacías

            if str(row[dia]).strip().upper() == "X":
                if hora_inicio is None:
                    hora_inicio = hora
            else:
                if hora_inicio is not None:
                    disponibilidad.append({
                        "CodDocente": cod_docente,
                        "Dia": dia,
                        "HoraInicio": f"{hora_inicio:02d}:00",
                        "HoraFin": f"{hora:02d}:00"
                    })
                    hora_inicio = None

        # Guardar el último rango si quedó abierto
        if hora_inicio is not None:
            disponibilidad.append({
                "CodDocente": cod_docente,
                "Dia": dia,
                "HoraInicio": f"{hora_inicio:02d}:00",
                "HoraFin": "22:00"
            })

    dataset_b = pd.DataFrame(disponibilidad)

    # Dataset C (fila 27 a 34 → índice 26:34), dropear col 2,3,4,5,8 → índice 1,2,3,4,7
    dataset_c_raw = df.iloc[26:32].copy()
    dataset_c = pd.concat([dataset_c_raw.iloc[:, [0]], dataset_c_raw.iloc[:, [5]]], axis=1)
    dataset_c.columns = ["Asignatura", "Escuela"]
    dataset_c.columns = ["Asignatura", "Escuela"]
    dataset_c["CodDocente"] = cod_docente


    return dataset_a, dataset_b, dataset_c

In [54]:
def procesar_carpeta(ruta_carpeta):
    archivos = [f for f in os.listdir(ruta_carpeta) if f.endswith(".xlsx")]
    
    todos_a, todos_b, todos_c = [], [], []

    for archivo in archivos:
        path = os.path.join(ruta_carpeta, archivo)
        dataset_a, dataset_b, dataset_c = procesar_archivo(path)
        todos_a.append(dataset_a)
        todos_b.append(dataset_b)
        todos_c.append(dataset_c)

    df_a_final = pd.concat(todos_a, ignore_index=True)
    df_b_final = pd.concat(todos_b, ignore_index=True)
    df_c_final = pd.concat(todos_c, ignore_index=True)

    return df_a_final, df_b_final, df_c_final

In [55]:
df_a, df_b, df_c = procesar_carpeta(CURRENT_DIR)

# Guardar si quieres
df_a.to_csv("datos_docentes.csv", index=False)
df_b.to_csv("disponibilidad_docentes.csv", index=False)
df_c.to_csv("cursos_preferencia_docentes.csv", index=False)


In [56]:
import unicodedata

def quitar_tildes(texto):
    if isinstance(texto, str):
        return unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')
    return texto




# Aplicar a todo el DataFrame

Se quita el formato utf-8 para facilitar cruces

In [57]:
## Preferencias docentes
df_format = df_c.applymap(quitar_tildes)

C:\Users\nicol\AppData\Local\Temp\ipykernel_27572\2679802552.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_format = df_c.applymap(quitar_tildes)


In [58]:
df_format.head()

,Asignatura,Escuela,CodDocente
0,CONTABILIDAD PARA LA GESTION,EP. SISTEMAS,41388541
1,CONTABILIDAD PARA LA GESTION,EP. SISTEMAS,41388541
2,CONTABILIDAD PARA LA GESTION,EP.SOFTWARE,41388541
3,CONTABILIDAD PARA LA GESTION,EP. CIENCIA DE LA COMPUTACION,41388541
4,SISTEMAS DIGITALES,EP. SISTEMAS,095699


# Homologamos el dataset cursos_preferencia_docentes.csv

In [59]:
EXTRACCION_DIR=(BASE_DIR / ".." / "Data" / "EXTRAIDOS").resolve()

In [60]:
EXTRACCION_DIR

WindowsPath('C:/Users/nicol/OneDrive/Documentos/GitHub/timeTablingFisi/Data/Extraidos')

In [61]:
ciclo=pd.read_csv((EXTRACCION_DIR/"ciclo_asignatura.csv").resolve(),sep=";",encoding="utf-8-sig")

In [62]:
#ciclo.reset_index(drop=True, inplace=True)

In [63]:
ciclo_format = ciclo.applymap(quitar_tildes)

C:\Users\nicol\AppData\Local\Temp\ipykernel_27572\3242078451.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ciclo_format = ciclo.applymap(quitar_tildes)


In [64]:
ciclo_format

,Ciclo,Asignatura,CodAsignatura,NomAsignatura
0,1,INE001 - PROCESO CULTURAL ANDINO,INE001,PROCESO CULTURAL ANDINO
1,1,INE002 - PROGRAMACION Y COMPUTACION,INE002,PROGRAMACION Y COMPUTACION
2,1,INE003 - DIBUJO TECNICO,INE003,DIBUJO TECNICO
3,1,INE004 - INGLES PARA ESCRITURA ACADEMICA,INE004,INGLES PARA ESCRITURA ACADEMICA
4,1,INE005 - MATLAB,INE005,MATLAB
...,...,...,...,...
83,10,202W1002 - DESARROLLO DE TESIS II,202W1002,DESARROLLO DE TESIS II
84,10,202W1003 - PRACTICA PRE PROFESIONAL,202W1003,PRACTICA PRE PROFESIONAL
85,10,202W1004 - TALLER DE APLICACIONES SOCIALES,202W1004,TALLER DE APLICACIONES SOCIALES
86,10,202W1005 - TENDENCIAS DE ARQUITECTURA DE SOFTWARE,202W1005,TENDENCIAS DE ARQUITECTURA DE SOFTWARE


In [65]:
df_merged = pd.merge(df_format, ciclo_format, left_on='Asignatura', right_on='NomAsignatura', how='left')


In [66]:
df_col=df_merged.drop(columns=["NomAsignatura","Asignatura_y","Ciclo"])

In [67]:
df_col = df_col.rename(columns={"Asignatura_x": "Asignatura"})

In [68]:
df_filtrado = df_col[df_col['Escuela'] == 'EP.SOFTWARE']

In [69]:
df_filtrado.reset_index(drop=True, inplace=True)

In [70]:
df_filtrado.head()

,Asignatura,Escuela,CodDocente,CodAsignatura
0,CONTABILIDAD PARA LA GESTION,EP.SOFTWARE,41388541,202W0402
1,SISTEMAS DIGITALES,EP.SOFTWARE,095699,202W0407
2,"INNOVACION, TECNOLOGIA Y EMPRENDIMIENTO",EP.SOFTWARE,0A4524,202W0403
3,PROBABILIDADES,EP.SOFTWARE,043788,202W0405
4,ALGORITMICA II,EP.SOFTWARE,95753,202W0401


In [71]:
df_filtrado.to_csv("..\\Data\\Extraidos\\cursos_preferencia_docentes_f.csv", sep=";",encoding="utf-8-sig",index=False)

# Generamos combinaciones disponibles de horario por docente

In [72]:
BASE_DIR

WindowsPath('C:/Users/nicol/OneDrive/Documentos/GitHub/timeTablingFisi/Extraccion')

In [73]:
# Cargar datasets
disponibilidad=pd.read_csv((BASE_DIR / ".." /"Extraccion" /"disponibilidad_docentes.csv").resolve(),sep=",",encoding="utf-8-sig")
preferencias=pd.read_csv("..\\Data\\Extraidos\\cursos_preferencia_docentes_f.csv",sep=";",encoding="utf-8-sig")
duraciones=pd.read_csv((EXTRACCION_DIR/"duracion_asignatura.csv").resolve(),sep=";",encoding="utf-8-sig")

In [74]:
# Asegurarse de que los nombres de columnas estén sin tildes
disponibilidad.columns = disponibilidad.columns.str.normalize("NFKD").str.encode("ascii", errors="ignore").str.decode("utf-8")
preferencias.columns = preferencias.columns.str.normalize("NFKD").str.encode("ascii", errors="ignore").str.decode("utf-8")
duraciones.columns = duraciones.columns.str.normalize("NFKD").str.encode("ascii", errors="ignore").str.decode("utf-8")


In [75]:
disponibilidad.head()

,CodDocente,Dia,HoraInicio,HoraFin
0,41388541,Lun,13:00,17:00
1,41388541,Lun,18:00,22:00
2,41388541,Mar,13:00,17:00
3,41388541,Mar,18:00,22:00
4,41388541,Mie,13:00,17:00


In [76]:
preferencias.head()

,Asignatura,Escuela,CodDocente,CodAsignatura
0,CONTABILIDAD PARA LA GESTION,EP.SOFTWARE,41388541,202W0402
1,SISTEMAS DIGITALES,EP.SOFTWARE,095699,202W0407
2,"INNOVACION, TECNOLOGIA Y EMPRENDIMIENTO",EP.SOFTWARE,0A4524,202W0403
3,PROBABILIDADES,EP.SOFTWARE,043788,202W0405
4,ALGORITMICA II,EP.SOFTWARE,95753,202W0401


In [77]:
duraciones.head()

,NomAsignatura,DuracionHoras,Ciclo,CodAsignatura
0,ALGORÍTMICA I,5.0,3,202W0301
1,ESTADÍSTICA,5.0,3,202W0302
2,FISICA ELÉCTRONICA,4.0,3,202W0303
3,INGENIERÍA ECONÓMICA,4.0,3,202W0304
4,INTRODUCCIÓN AL DESARROLLO DE SOFTWARE,4.0,3,202W0305


In [78]:
disponibilidad["key"] = 1
preferencias["key"] = 1

cruce = pd.merge(disponibilidad, preferencias, on=["CodDocente", "key"]).drop(columns="key")

In [79]:
cruce

,CodDocente,Dia,HoraInicio,HoraFin,Asignatura,Escuela,CodAsignatura
0,41388541,Lun,13:00,17:00,CONTABILIDAD PARA LA GESTION,EP.SOFTWARE,202W0402
1,41388541,Lun,18:00,22:00,CONTABILIDAD PARA LA GESTION,EP.SOFTWARE,202W0402
2,41388541,Mar,13:00,17:00,CONTABILIDAD PARA LA GESTION,EP.SOFTWARE,202W0402
3,41388541,Mar,18:00,22:00,CONTABILIDAD PARA LA GESTION,EP.SOFTWARE,202W0402
4,41388541,Mie,13:00,17:00,CONTABILIDAD PARA LA GESTION,EP.SOFTWARE,202W0402
5,41388541,Mie,18:00,22:00,CONTABILIDAD PARA LA GESTION,EP.SOFTWARE,202W0402
6,41388541,Jue,10:00,15:00,CONTABILIDAD PARA LA GESTION,EP.SOFTWARE,202W0402
7,41388541,Jue,18:00,22:00,CONTABILIDAD PARA LA GESTION,EP.SOFTWARE,202W0402
8,41388541,Vie,13:00,17:00,CONTABILIDAD PARA LA GESTION,EP.SOFTWARE,202W0402
9,41388541,Vie,18:00,22:00,CONTABILIDAD PARA LA GESTION,EP.SOFTWARE,202W0402


In [80]:
disponibilidad.head()

,CodDocente,Dia,HoraInicio,HoraFin,key
0,41388541,Lun,13:00,17:00,1
1,41388541,Lun,18:00,22:00,1
2,41388541,Mar,13:00,17:00,1
3,41388541,Mar,18:00,22:00,1
4,41388541,Mie,13:00,17:00,1


In [81]:
preferencias.head()

,Asignatura,Escuela,CodDocente,CodAsignatura,key
0,CONTABILIDAD PARA LA GESTION,EP.SOFTWARE,41388541,202W0402,1
1,SISTEMAS DIGITALES,EP.SOFTWARE,095699,202W0407,1
2,"INNOVACION, TECNOLOGIA Y EMPRENDIMIENTO",EP.SOFTWARE,0A4524,202W0403,1
3,PROBABILIDADES,EP.SOFTWARE,043788,202W0405,1
4,ALGORITMICA II,EP.SOFTWARE,95753,202W0401,1


In [82]:
preferencias

,Asignatura,Escuela,CodDocente,CodAsignatura,key
0,CONTABILIDAD PARA LA GESTION,EP.SOFTWARE,41388541,202W0402,1
1,SISTEMAS DIGITALES,EP.SOFTWARE,095699,202W0407,1
2,"INNOVACION, TECNOLOGIA Y EMPRENDIMIENTO",EP.SOFTWARE,0A4524,202W0403,1
3,PROBABILIDADES,EP.SOFTWARE,043788,202W0405,1
4,ALGORITMICA II,EP.SOFTWARE,95753,202W0401,1
5,MATEMATICA DISCRETA,EP.SOFTWARE,0A1610,202W0404,1


In [83]:
cruce = cruce.merge(duraciones[["CodAsignatura", "DuracionHoras","Ciclo"]], on="CodAsignatura", how="left")

In [84]:
cruce

,CodDocente,Dia,HoraInicio,HoraFin,Asignatura,Escuela,CodAsignatura,DuracionHoras,Ciclo
0,41388541,Lun,13:00,17:00,CONTABILIDAD PARA LA GESTION,EP.SOFTWARE,202W0402,4.0,4
1,41388541,Lun,18:00,22:00,CONTABILIDAD PARA LA GESTION,EP.SOFTWARE,202W0402,4.0,4
2,41388541,Mar,13:00,17:00,CONTABILIDAD PARA LA GESTION,EP.SOFTWARE,202W0402,4.0,4
3,41388541,Mar,18:00,22:00,CONTABILIDAD PARA LA GESTION,EP.SOFTWARE,202W0402,4.0,4
4,41388541,Mie,13:00,17:00,CONTABILIDAD PARA LA GESTION,EP.SOFTWARE,202W0402,4.0,4
5,41388541,Mie,18:00,22:00,CONTABILIDAD PARA LA GESTION,EP.SOFTWARE,202W0402,4.0,4
6,41388541,Jue,10:00,15:00,CONTABILIDAD PARA LA GESTION,EP.SOFTWARE,202W0402,4.0,4
7,41388541,Jue,18:00,22:00,CONTABILIDAD PARA LA GESTION,EP.SOFTWARE,202W0402,4.0,4
8,41388541,Vie,13:00,17:00,CONTABILIDAD PARA LA GESTION,EP.SOFTWARE,202W0402,4.0,4
9,41388541,Vie,18:00,22:00,CONTABILIDAD PARA LA GESTION,EP.SOFTWARE,202W0402,4.0,4


In [85]:
from datetime import datetime, timedelta

def generar_franjas(hora_inicio, hora_fin, duracion_horas):
    formato = "%H:%M"
    h_inicio = datetime.strptime(hora_inicio, formato)
    h_fin = datetime.strptime(hora_fin, formato)
    resultado = []

    while h_inicio + timedelta(hours=duracion_horas) <= h_fin:
        nueva_fin = h_inicio + timedelta(hours=duracion_horas)
        resultado.append((h_inicio.strftime(formato), nueva_fin.strftime(formato)))
        h_inicio += timedelta(hours=1)  # ← solo se avanza en horas completas

    return resultado

In [86]:
def expandir_franjas(row):
    franjas = generar_franjas(row["HoraInicio"], row["HoraFin"], row["DuracionHoras"])
    return pd.DataFrame({
        "CodDocente": row["CodDocente"],
        "Día": row["Dia"],
        "Asignatura": row["Asignatura"],
        "CodAsignatura": row["CodAsignatura"],
        "DuracionHoras": row["DuracionHoras"],
        "HoraInicio": [ini for ini, _ in franjas],
        "HoraFin": [fin for _, fin in franjas],
        "Ciclo" : row["Ciclo"]
    })

# Aplicamos fila por fila
posibles_horarios = pd.concat([expandir_franjas(row) for _, row in cruce.iterrows()], ignore_index=True)

In [87]:
posibles_horarios

,CodDocente,Día,Asignatura,CodAsignatura,DuracionHoras,HoraInicio,HoraFin,Ciclo
0,41388541,Lun,CONTABILIDAD PARA LA GESTION,202W0402,4.0,13:00,17:00,4
1,41388541,Lun,CONTABILIDAD PARA LA GESTION,202W0402,4.0,18:00,22:00,4
2,41388541,Mar,CONTABILIDAD PARA LA GESTION,202W0402,4.0,13:00,17:00,4
3,41388541,Mar,CONTABILIDAD PARA LA GESTION,202W0402,4.0,18:00,22:00,4
4,41388541,Mie,CONTABILIDAD PARA LA GESTION,202W0402,4.0,13:00,17:00,4
...,...,...,...,...,...,...,...,...
106,0A1610,Vie,MATEMATICA DISCRETA,202W0404,4.0,14:00,18:00,4
107,0A1610,Vie,MATEMATICA DISCRETA,202W0404,4.0,15:00,19:00,4
108,0A1610,Vie,MATEMATICA DISCRETA,202W0404,4.0,16:00,20:00,4
109,0A1610,Vie,MATEMATICA DISCRETA,202W0404,4.0,17:00,21:00,4


In [88]:
posibles_horarios.to_csv("..\\Data\\Extraidos\\disponibilidad_docentes_real.csv", sep=",",index=False)